# Defining Release Scenarios

The whole point of `GasDispersion.jl` is to model chemical releases into the atmosphere, to do this one needs to have a defined chemical release scenario. Unless otherwise specified, the release is treated as a jet release at a given release height, emitting a constant mass emission rate for a given release duration. The coordinate system is centered at the releasse point and at ground level, that is the release point is at *x=0*, *y=0*, *z=h* where *x* is the downwind distance, *y* the cross wind distance, and *z* the elevation. The gas dispersion models require the following parameters to be defined for a given scenario:

- mass emission rate, kg/s
- release duration, s
- jet diameter, m
- jet velocity, m/s
- jet density, kg/m³
- release pressure, Pa
- release temperature, K
- release height, m
- windspeed, m/s
- ambient density, kg/m³
- ambient pressure, Pa
- ambient temperature, K
- pasquill stability class

Standard methods given in *Lee's Loss Prevention in the Process Industries*, the CCPS texts such as *Guidelines for Consequence Analysis of Chemical Releases*, *Guidelines for Chemical Process Quantitative Risk Analysis*, and *Guidelines for Vapour Cloud Explosion, Pressure Vessel Burst, BLEVE and Flash Fire Hazards*, and many other texts on chemical proess safety can be used to determine all of these parameters and a `Scenario` created directly.

In [1]:
using GasDispersion

In [2]:
scenario = Scenario(
    1.0,   # mass emission rate, kg/s
    10.0,  # release duration, s
    0.25,  # jet diameter, m
    15.67, # jet velocity, m/s
    1.3,   # jet density, kg/m³
    101325,# release_pressure, Pa
    450,   # release temperature, K
    1.0,   # release height, m
    1.5,   # windspeed, m/s
    1.225, # ambient density, kg/m³
    101325,# ambient pressure, Pa
    298.15,# ambient temperature, K
    "F",   # pasquill stability class
)

Release scenario:
    mass_emission_rate: 1.0 kg/s 
    release_duration: 10.0 s 
    jet_diameter: 0.25 m 
    jet_velocity: 15.67 m/s 
    jet_density: 1.3 kg/m^3 
    release_pressure: 101325 Pa 
    release_temperature: 450 K 
    release_height: 1.0 m 
    windspeed: 1.5 m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: F  
    

Or by passing a dictionary with the known values (missing values will be set to `missing`)

In [3]:
d = Dict([
    :release_pressure => 101325,   # Pa
    :release_temperature => 298.15,# K
    :ambient_density => 1.225,     # kg/m^3
    :ambient_pressure => 101325,   # Pa
    :ambient_temperature => 298.15,# K
])

scenario = Scenario(d)

Release scenario:
    mass_emission_rate: missing kg/s 
    release_duration: missing s 
    jet_diameter: missing m 
    jet_velocity: missing m/s 
    jet_density: missing kg/m^3 
    release_pressure: 101325.0 Pa 
    release_temperature: 298.15 K 
    release_height: missing m 
    windspeed: missing m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: missing  
    

Or by passing an existing scenario, and replacing some parameters using keyword arguments

In [4]:
scenario = Scenario(scenario; mass_emission_rate=1.0, release_duration=10.0)

Release scenario:
    mass_emission_rate: 1.0 kg/s 
    release_duration: 10.0 s 
    jet_diameter: missing m 
    jet_velocity: missing m/s 
    jet_density: missing kg/m^3 
    release_pressure: 101325.0 Pa 
    release_temperature: 298.15 K 
    release_height: missing m 
    windspeed: missing m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: missing  
    

`GasDispersion` also comes with some basic tools for constructing simple release scenarios for common situations

## Discharge Through a Hole
The easiest is for a discharge through a hole, for example a puncture in the side of a process vessel.

### Liquid Discharge
Liquid discharge through a small hole can be modeled by the Bernoulli equation, with the jet velocity given by

$$ u_j = c_d  \sqrt{ 2 \left( P_r - P_a \over \rho_l \right) } $$

where
- $u_j$ is the jet velocity, m/s
- $c_d$ the discharge coefficient, unitless
- $P_r$ the release pressure, Pa, i.e. the pressure inside the vessel at the hole
- $P_a$ the ambient pressure, Pa, i.e. the pressure outside the hole
- $\rho_l$ the liquid density, kg/m³

the mass emission rate is then given by
$$ \dot{m} = \rho_l A_h u_j $$

where $A_h$ is the cross-sectional area of the hole, in m²

#### Example

Suppose the scenario, identified through a hazard analysis, is for a leak of a liquid from a storage tank through a 10mm hole. The tank headspace is at 0.1 barg and there is 2 m of liquid head above the hole, and the liquid has a density of 490 kg/m³, and is in thermal equilibrium with the environment. Additionally, let's assue a discharge coefficient of 0.63 and a circular hole.

For ambient conditions we assume the atmosphere is at ambient conditions of 1atm and 25°C, with a windspeed of 1.5m/s and class F stability.

Assumptions:
- $c_d = 0.63 $
- $D_j = 0.010 \textrm{m}$
- $\rho_j = \rho_l = 490 \textrm{kg/m}^3$
- $T_r = 298.15 \textrm{K}$
- $P_a = 101325 \textrm{Pa}$
- $T_a = 298.15 \textrm{K}$
- $\rho_a = 1.225 \textrm{kg/m}^3$
- $u = 1.5 \textrm{m/s}$

What we need to solve for is
- $P_r$, the release pressure, Pa,
- $u_j$, the jet velocity, m/s
- $\dot{m}$, the mass emission rate, kg/s

In [5]:
# What we know
g = 9.80616 #m/s²
cd = 0.63
Dⱼ = 0.010 #m
ρⱼ = 490.0 #kg/m³
Tᵣ = 298.15 #K
Pₐ = 101325.0 #Pa
Tₐ = 298.15 #K
ρₐ = 1.225 #kg/m³
u = 1.5 #m/s
stability = "F"

"F"

The pressure immediately inside the liquid discharge is the sum of the headspace pressure and the pressure due to the 2 m of liquid head and is

$$ P_r = P_{headspace} + \rho_j g h $$
$$ P_r = (10000 Pa + 101325 Pa) + 490 kg/m³ \cdot 9.80616 m/s² \cdot 2 m $$

In [6]:
Pᵣ = (10000 + Pₐ) + ρⱼ*g*2

120935.0368

In [7]:
uⱼ = cd * √( 2*( (Pᵣ - Pₐ) / ρⱼ) )

Aₕ = (π/4)*Dⱼ^2

m = ρⱼ*Aₕ*uⱼ

0.21691154763598

In [8]:
s = Dict([
    :mass_emission_rate => m,
    :jet_diameter => Dⱼ,
    :jet_velocity => uⱼ,
    :jet_density => ρⱼ,
    :release_pressure => Pᵣ,
    :release_temperature => Tᵣ,
    :release_height => 0.0,
    :windspeed => u,
    :ambient_density => ρₐ,
    :ambient_pressure => Pₐ,
    :ambient_temperature => Tₐ,
    :pasquill_gifford => stability
])

scenario_1 = Scenario(s)

Release scenario:
    mass_emission_rate: 0.21691154763598 kg/s 
    release_duration: missing s 
    jet_diameter: 0.01 m 
    jet_velocity: 5.636333880812954 m/s 
    jet_density: 490.0 kg/m^3 
    release_pressure: 120935.0368 Pa 
    release_temperature: 298.15 K 
    release_height: 0.0 m 
    windspeed: 1.5 m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: F  
    

We can use the function `scenario_builder` to help simplify this, using default values for ambient conditions and specifying some parameters for the release scenario.

In [9]:
scenario_2 = scenario_builder(Pᵣ, Tᵣ; windspeed=u, stability=stability, 
             model="jet", phase="liquid", liquid_density=ρⱼ, hole_diameter=Dⱼ, discharge_coeff=0.63)

Release scenario:
    mass_emission_rate: 0.21691154763598 kg/s 
    release_duration: missing s 
    jet_diameter: 0.01 m 
    jet_velocity: 5.636333880812954 m/s 
    jet_density: 490.0 kg/m^3 
    release_pressure: 120935.0368 Pa 
    release_temperature: 298.15 K 
    release_height: 0.0 m 
    windspeed: 1.5 m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: F  
    

In [10]:
scenario_1 == scenario_2

true

### Gas Discharge

TODO

### Two-Phase Discharge

TODO